# Mechelse Heide Zuid (MHZ) quarry - preprocessing - cleaned

## Imports

In [1]:
# Scientific packages
import pandas as pd
import numpy as np

# Operating system interaction package
import os

# Visualisation packages
import seaborn as sns
import matplotlib.pyplot as plt

# Coordinate handling package
import pyproj

# Geographical map package
import folium
from folium.features import DivIcon

# Local package with project specific functions
from borehole_analysis import processing as bap

In [2]:
# Load jupyter extension to reload packages before executing user code.
# This is useful for local packages when code is still being changed
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload

In [3]:
# Reload all packages (except those excluded by %aimport) every time before executing the Python code typed.
%autoreload 2

## Files

1. collar 85 boreholes
2. survey 83 boreholes
3. code quality 2008 38 boreholes
4. geology 57 boreholes
5. granulo 82 boreholes
6. granulo backup without codes 78 boreholes
7. washed 8 boreholes
8. styles

In [4]:
# Set coordinates projections
lam72 = pyproj.Proj(init="ESRI:103300") # Lambert 72
wgs84 = pyproj.Proj(init="EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth

In [5]:
path_MHZ = "../_DATA/MHZ_groeve/"

___

## 1&2. MHZ collar and survey

In [6]:
df_MHZ_collar = pd.read_csv(path_MHZ + "MHZ_collar.csv", sep=";") # Basic information
df_MHZ_survey = pd.read_csv(path_MHZ + "MHZ_survey.csv", sep=";") # Doesn't add new information
df_MHZ_collar.tail()

,hole_id,hole_path,max_depth,x,y,z,depth_water_table,date,drilling_company,Project,comment
80,MHZ_16_04,NaN,35.0,237482.70,187585.60,78.82,5.5,24/5/2016 0:00:00,GEOTRON,PROJ-009323,NaN
81,MHZ_16_05,NaN,35.0,237238.80,187649.60,78.74,4.0,3/6/2016 0:00:00,GEOTRON,PROJ-009323,NaN
82,MHZ_16_06,NaN,35.0,237416.70,187492.50,76.80,6.0,23/5/2016 0:00:00,GEOTRON,PROJ-009323,NaN
83,MHZ_16_07,NaN,35.0,237224.95,187521.48,78.69,10.0,6/6/2016 0:00:00,GEOTRON,PROJ-009323,NaN
84,MHZ_16_08,NaN,35.0,237541.40,187315.00,74.30,5.0,11/5/2016 0:00:00,GEOTRON,PROJ-009323,NaN


In [7]:
# Transform coordinates from Lambert72 to WGS84
MHZ_lon, MHZ_lat = pyproj.transform(lam72, wgs84, df_MHZ_collar["x"].values, df_MHZ_collar["y"].values)

In [8]:
# Change Lambert72 coordinates to WGS84 coordinates in degrees
df_MHZ_collar["x"] = MHZ_lat
df_MHZ_collar["y"] = MHZ_lon

# Rename columns
df_MHZ_collar.rename(columns={"x" : "lat", 
                              "y" : "lon", 
                              "z" : "elevation"}, inplace=True)

# Drop unnecessary columns
df_MHZ_collar.drop(["hole_path"], axis=1, inplace=True)
df_MHZ_collar.tail()

,hole_id,max_depth,lat,lon,elevation,depth_water_table,date,drilling_company,Project,comment
80,MHZ_16_04,35.0,50.992436,5.613555,78.82,5.5,24/5/2016 0:00:00,GEOTRON,PROJ-009323,NaN
81,MHZ_16_05,35.0,50.993048,5.610096,78.74,4.0,3/6/2016 0:00:00,GEOTRON,PROJ-009323,NaN
82,MHZ_16_06,35.0,50.991609,5.612592,76.80,6.0,23/5/2016 0:00:00,GEOTRON,PROJ-009323,NaN
83,MHZ_16_07,35.0,50.991898,5.609869,78.69,10.0,6/6/2016 0:00:00,GEOTRON,PROJ-009323,NaN
84,MHZ_16_08,35.0,50.989995,5.614326,74.30,5.0,11/5/2016 0:00:00,GEOTRON,PROJ-009323,NaN


In [9]:
df_MHZ_collar["date"] = pd.to_datetime(df_MHZ_collar['date'], dayfirst=True)

In [10]:
df_MHZ_collar["year"] = df_MHZ_collar["date"].map(lambda x: x.year)
df_MHZ_collar.tail()

,hole_id,max_depth,lat,lon,elevation,depth_water_table,date,drilling_company,Project,comment,year
80,MHZ_16_04,35.0,50.992436,5.613555,78.82,5.5,2016-05-24,GEOTRON,PROJ-009323,NaN,2016
81,MHZ_16_05,35.0,50.993048,5.610096,78.74,4.0,2016-06-03,GEOTRON,PROJ-009323,NaN,2016
82,MHZ_16_06,35.0,50.991609,5.612592,76.80,6.0,2016-05-23,GEOTRON,PROJ-009323,NaN,2016
83,MHZ_16_07,35.0,50.991898,5.609869,78.69,10.0,2016-06-06,GEOTRON,PROJ-009323,NaN,2016
84,MHZ_16_08,35.0,50.989995,5.614326,74.30,5.0,2016-05-11,GEOTRON,PROJ-009323,NaN,2016


In [11]:
# Drop rows with no coordinate
df_MHZ_collar = df_MHZ_collar.drop(list(df_MHZ_collar[df_MHZ_collar["lat"].isnull()].index.values))

In [12]:
# Create map
MHZ_map = folium.Map()

# Add boreholes to map
for index, row in df_MHZ_collar.iterrows():
    # Create borehole markers
    marker = folium.features.Circle(radius=5, 
                                    location=[row["lat"], row["lon"]], 
                                    color='crimson',
                                    fill=True,
                                    fill_color='crimson',
                                    fill_opacity=1)
    # Add borehole markers to map
    marker.add_to(MHZ_map)
    
    # Create borehole labels
    text = folium.Marker(location=[row["lat"], row["lon"]], 
                         popup="year: " + str(row["year"]),
                         icon=DivIcon(icon_size=(150,36), 
                                      icon_anchor=(-1,-1), 
                                      html='<div style="font-size: 8pt">' + row["hole_id"] +'</div>'))
    # Add borehole labels to map
    text.add_to(MHZ_map)

# Map bounding coordinates
lon_max, lat_max= df_MHZ_collar["lon"].max(), df_MHZ_collar["lat"].max()
lon_min, lat_min =  df_MHZ_collar["lon"].min(), df_MHZ_collar["lat"].min()   
    
# Set bounds of map to display
MHZ_map.fit_bounds([(lat_max, lon_max), (lat_min, lon_min)])

# Show map
MHZ_map

In [12]:
df_MHZ_collar.to_csv("..\_RESULTS\coordinatesMHZ.csv", index=False)

___

### 3. MHZ code quality 2008

In [13]:
df_MHZ_code_quality_2008 = pd.read_csv(path_MHZ + "MHZ_code_quality_2008.csv", sep=";")
df_MHZ_code_quality_2008.drop(["samp_id"], inplace=True, axis=1) # empty column
df_MHZ_code_quality_2008.tail()

,hole_id,depth_from,depth_to,quality_code100,quality_code110
1715,MHZ_08_05,24,25,3.3,3.3
1716,MHZ_08_05,25,26,3.3,3.3
1717,MHZ_08_05,26,27,3.3,3.3
1718,MHZ_08_05,27,28,3.3,3.3
1719,MHZ_08_05,28,29,4.0,4.0


In [14]:
df_MHZ_code_quality_2008["hole_id"].unique()

array(['LBU_05_01', 'LBU_05_02', 'LBU_05_03', 'LBU_05_04', 'LBU_05_05',
       'LBU_05_06', 'LBU_05_07', 'LBU_05_08', 'LBU_05_09', 'LBU_05_10',
       'LBU_05_11', 'LBU_05_12', 'LBU_05_13', 'LBU_05_14', 'LBU_05_15',
       'LBU_05_16', 'LBU_05_17', 'LBU_05_18', 'LBU_05_19', 'LBU_05_20',
       'LBU_05_21', 'LBU_05_22', 'LBU_05_23', 'LBU_05_24', 'LBU_05_25',
       'LBU_05_26', 'LBU_05_27', 'LBU_05_28', 'LBU_05_29', 'LBU_05_30',
       'LBU_07_01', 'LBU_07_02', 'LBU_07_03', 'MHZ_08_01', 'MHZ_08_02',
       'MHZ_08_03', 'MHZ_08_04', 'MHZ_08_05'], dtype=object)

___

### 4. MHZ geology

In [15]:
df_MHZ_geology = pd.read_csv(path_MHZ + "MHZ_geology.csv", sep=";", engine="python")
df_MHZ_geology.tail()

,hole_id,depth_from,depth_to,description_flemish,description_english,litho,color,color_simple,organic_matter,code
778,MHZ_12_04,18.0,27.0,"Zand, matig fijn, zwak siltig, zwak veenhouden...","Sand, moderately fine, silty weak, weak peat c...",sand,dark brown beige,dark brown,weak peat,0
779,MHZ_12_04,27.0,29.0,"Zand, matig fijn, zwak siltig, resten kalk, zw...","Sand , finely weak silty , lime content, weak ...",sand,white beige,white beige,weak wood,0
780,MHZ_12_04,29.0,30.5,"Zand, matig fijn, zwak siltig, resten kalk, zw...","Sand, finely weak silty, lime residue, weak w...",sand,white beige,white beige,weak wood,0
781,MHZ_12_04,30.5,35.0,"Zand, zeer fijn, zwak siltig, resten kalk, zwa...","Sand, very fine, weak silty , lime residue, we...",sand,white beige,white beige,weak wood,0
782,MHZ_16_01,0.0,8.9,"Leem, vast, matig zandig, sterk\ngrindhoudend,...",NaN,NaN,NaN,brown,NaN,0


In [16]:
df_MHZ_geology["color_simple"].value_counts().count()

27

___

### 5. MHZ granulo

In [17]:
df_MHZ_granulo = pd.read_csv(path_MHZ + "MHZ_granulo.csv", sep=";")
df_MHZ_granulo.tail()

,hole_id,depth_from,depth_to,samp_id,z_1000,z_710,z_500,z_355,z_250,z_180,...,z_90,z_63,z_0,afa,ags,theor_spec_surface,fineness_modulus,clay,code_geol,Comment
2376,MHZ_16_08,30.0,31.0,PR-016635-066,0.0,0.0,0.5,2.1,4.3,31.6,...,7.0,0.7,0.4,75.0,182.0,137.0,1.00,0.44,NaN,NaN
2377,MHZ_16_08,31.0,32.0,PR-016635-067,0.0,0.0,0.1,0.6,2.7,27.1,...,9.4,1.0,0.4,79.0,170.0,143.0,0.93,0.40,NaN,NaN
2378,MHZ_16_08,32.0,33.0,PR-016635-068,0.0,0.0,0.3,2.0,3.8,29.5,...,8.0,0.8,0.3,77.0,179.0,139.0,0.97,0.68,NaN,NaN
2379,MHZ_16_08,33.0,34.0,PR-016635-069,0.0,0.0,0.0,1.0,4.3,28.7,...,8.0,0.8,0.3,77.0,175.0,140.0,0.96,0.51,NaN,NaN
2380,MHZ_16_08,34.0,35.0,PR-016635-070,0.0,0.0,0.1,1.6,4.2,28.0,...,8.7,0.9,0.4,78.0,176.0,141.0,0.96,0.81,NaN,NaN


In [18]:
# Samples with GSD proportion sum higher than 100
check_sum = df_MHZ_granulo.loc[:, ["hole_id", "z_1000", "z_710", "z_500", "z_355", "z_250", "z_180", "z_125", "z_90", "z_63", "z_0"]]
check_sum[(check_sum.sum(axis=1) > 101)]

,hole_id,z_1000,z_710,z_500,z_355,z_250,z_180,z_125,z_90,z_63,z_0
1660,LBU_87_1,0.0,0.0,0.2,40.6,32.5,44.6,7.4,4.1,0.7,0.0
1661,LBU_87_1,0.0,0.0,0.2,40.6,32.5,44.6,7.4,4.1,0.7,0.0
1734,LBU_87_7,0.0,0.9,13.5,23.8,30.2,29.2,4.2,2.2,2.2,0.0


In [19]:
# Samples with GSD proportion lower than 100
check_sum[((check_sum.sum(axis=1) < 99) & (check_sum.sum(axis=1) > 50))]

,hole_id,z_1000,z_710,z_500,z_355,z_250,z_180,z_125,z_90,z_63,z_0
1690,LBU_87_3,0.0,0.0,0.0,1.1,21.2,55.2,17.6,2.0,0.1,1.7
1692,LBU_87_3,0.0,0.0,0.1,6.2,33.3,43.6,12.9,2.0,0.1,NaN
1693,LBU_87_3,0.0,0.0,0.1,6.2,33.3,43.6,12.9,2.0,0.1,-1.0


In [20]:
# Check for negative values
for cols in check_sum.columns.tolist()[1:]:
    data = check_sum.loc[check_sum[cols] < 0]
data

,hole_id,z_1000,z_710,z_500,z_355,z_250,z_180,z_125,z_90,z_63,z_0
1693,LBU_87_3,0.0,0.0,0.1,6.2,33.3,43.6,12.9,2.0,0.1,-1.0


In [21]:
# Number of boreholes with code_geol information in granulo dataframe
len(df_MHZ_granulo[df_MHZ_granulo["code_geol"].notnull()]["hole_id"].unique())

70

Boreholes without code_geol info in granulo dataframe:  
Possibilities for code_geol completion exist, however (between brackets)

- LBU_01_2 (OVB)
- LBU_01_3 (OVB)
- LBU_02_1 (GZ)
- LBU_02_2 (GZ)
- LBU_03_01 (NO INFO)
- LBU_03_02 (NO INFO)
- LBU_87_1 (OVB, TZ)
- LBU_87_2 (OVB, TZ)
- LBU_87_3 (OVB, TZ)
- LBU_87_4 (OVB, TZ)
- LBU_87_5 (OVB, TZ)
- LBU_87_6 (OVB, TZ)
- LBU_87_7 (NO INFO)
- LBU_87_8 (NO INFO)
- LBU_96_1 (OVB, TZ)
- LBU_96_2 (OVB, TZ)
- LBU_96_3 (OVB, TZ)
- LBU_96_4 (OVB, TZ)
- LBU_98_1 (OVB, TZ)
- LBU_98_2 (OVB, TZ)
- LBU_98_4 (OVB, TZ)
- LBU_98_5 (OVB, TZ)
- LBU_98_6 (OVB, TZ)
- LBU_98_7 (OVB, TZ)
- MHZ_12_02 (OVB)
- MHZ_16_01 (NO INFO)
- MHZ_16_02 (NO INFO)
- MHZ_16_03 (NO INFO)
- MHZ_16_04 (NO INFO)
- MHZ_16_05 (NO INFO)
- MHZ_16_06 (NO INFO)
- MHZ_16_07 (NO INFO)
- MHZ_16_08 (NO INFO)

OK: 
- LBU_05_01 (OVB) OK from code_quality
- LBU_05_09 (OVB) OK from code_quality
- LBU_05_11 (OVB) OK from code_quality
- LBU_05_23 (OVB) OK from code_quality
- LBU_05_24 (OVB) OK from code_quality
- LBU_07_02 (OVB) OK from code_quality

In [22]:
# List with borehole ids
hole_ids = df_MHZ_granulo["hole_id"].unique()

# Initialize boreholes with/without GSD lists
hole_ids_withoutGSD = []
hole_ids_withGSD = []

# Loop through every borehole in washed dataframe
for hole_id in hole_ids:
    hole = df_MHZ_granulo[df_MHZ_granulo["hole_id"] == hole_id]
    
    # Check if all GSD for z_1000 of borehole are NaN
    # append borehole to withoutGSD if True
    if hole["z_1000"].isnull().sum() == hole.shape[0]:
        hole_ids_withoutGSD.append(hole_id)
        
    # append borehole to withGSD if False
    else:
        hole_ids_withGSD.append(hole_id)
       
print("There are", len(hole_ids_withGSD), "boreholes with GSD data")
print("There are", len(hole_ids_withoutGSD), "boreholes without GSD data")

There are 76 boreholes with GSD data
There are 6 boreholes without GSD data


In [22]:
# Convert code_geol using dict
code_geol_dict = {
    1.0 : "OVB",
    2.0 : "TZ",
    3.0 : "IZ",
    4.0 : "GZ"
}

df_MHZ_granulo["code_geol"] = df_MHZ_granulo["code_geol"].replace(code_geol_dict)
df_MHZ_granulo["code_geol"].value_counts()

GZ     808
IZ     689
TZ     424
OVB      7
Name: code_geol, dtype: int64

In [23]:
# Create subsets based on code_geol
subsets_MHZ_granulo = bap.subset_df_based_on_categories(df_MHZ_granulo, "code_geol")

# Delete key with entries without code_geol info
del subsets_MHZ_granulo[np.nan]

In [24]:
subsets_MHZ_averages_granulo  = {}

for cat, df in subsets_MHZ_granulo.items():
    subsets_MHZ_averages_granulo[cat] = bap.combine_id_rows_complex(df, cat, "hole_id", "depth_from", "depth_to")

TZ merge: (49, 19)
IZ merge: (68, 19)
GZ merge: (65, 19)
OVB merge: (6, 19)


In [25]:
# Check that averaged GSD data sum up to 100 per code_geol per hole_id
checker = subsets_MHZ_averages_granulo["IZ"].loc[:, ["hole_id","code_geol", "z_1000", "z_710", "z_500", "z_355", "z_250", "z_180", "z_125", "z_90", "z_63", "z_0"]]
checker.set_index("hole_id")

#checker[np.isclose(checker.sum(axis=1), 100, atol=0.2)].count()
checker[checker.sum(axis=1) > 101].sum(axis=1)

43    107.6
dtype: float64

In [26]:
subsets_MHZ_averages_granulo["IZ"].head()

,hole_id,depth_from,depth_to,code_geol,z_1000,z_710,z_500,z_355,z_250,z_180,z_125,z_90,z_63,z_0,afa,ags,theor_spec_surface,fineness_modulus,clay
0,LBU_01_1,23.94,34.08,IZ,0.050000,0.025000,0.425000,4.908333,27.308333,50.350000,12.500000,2.783333,1.450000,0.175000,61.416667,238.416667,NaN,NaN,NaN
1,LBU_01_2,27.25,35.00,IZ,0.000000,0.000000,0.225000,3.725000,30.575000,52.562500,11.437500,0.950000,0.262500,0.187500,58.625000,241.625000,NaN,NaN,NaN
2,LBU_01_3,28.38,35.00,IZ,0.014286,0.014286,0.557143,6.471429,34.571429,44.571429,12.028571,1.200000,0.285714,0.185714,57.428571,251.428571,NaN,NaN,NaN
3,LBU_02_3,22.00,35.00,IZ,0.000000,0.000000,0.250000,5.250000,38.450000,46.350000,9.050000,0.425000,0.125000,0.050000,55.500000,254.500000,NaN,NaN,NaN
4,LBU_02_4,20.00,34.00,IZ,0.100000,0.000000,0.450000,3.375000,21.450000,53.975000,19.775000,0.525000,0.200000,0.050000,61.500000,230.000000,NaN,NaN,NaN


In [27]:
gsd_columns = ["z_1000", "z_710", "z_500", "z_355", "z_250", "z_180", "z_125", "z_90", "z_63", "z_0"]

In [28]:
# Recalculate proportions to 100%
subsets_MHZ_averages_granulo_norm = {}

for cat, df in subsets_MHZ_averages_granulo.items():
    df_norm = df.copy()
    df_norm.loc[:, gsd_columns] = df_norm.loc[:, gsd_columns].divide(df_norm.loc[:, gsd_columns].sum(axis=1), axis=0) * 100
    subsets_MHZ_averages_granulo_norm[cat] = df_norm

In [29]:
subsets_MHZ_averages_granulo_norm["IZ"].head()

,hole_id,depth_from,depth_to,code_geol,z_1000,z_710,z_500,z_355,z_250,z_180,z_125,z_90,z_63,z_0,afa,ags,theor_spec_surface,fineness_modulus,clay
0,LBU_01_1,23.94,34.08,IZ,0.050013,0.025006,0.425106,4.909561,27.315162,50.362591,12.503126,2.784029,1.450363,0.175044,61.416667,238.416667,NaN,NaN,NaN
1,LBU_01_2,27.25,35.00,IZ,0.000000,0.000000,0.225169,3.727796,30.597948,52.601951,11.446085,0.950713,0.262697,0.187641,58.625000,241.625000,NaN,NaN,NaN
2,LBU_01_3,28.38,35.00,IZ,0.014300,0.014300,0.557701,6.477906,34.606035,44.616045,12.040612,1.201201,0.286000,0.185900,57.428571,251.428571,NaN,NaN,NaN
3,LBU_02_3,22.00,35.00,IZ,0.000000,0.000000,0.250125,5.252626,38.469235,46.373187,9.054527,0.425213,0.125063,0.050025,55.500000,254.500000,NaN,NaN,NaN
4,LBU_02_4,20.00,34.00,IZ,0.100100,0.000000,0.450450,3.378378,21.471471,54.029029,19.794795,0.525526,0.200200,0.050050,61.500000,230.000000,NaN,NaN,NaN


In [30]:
# Recombine subsets
recombined = pd.concat(list(subsets_MHZ_averages_granulo_norm.values()), axis=0)

# Only one value per groupby so mean just represents present value
recombined_granulo = recombined.groupby(["hole_id", "code_geol"]).mean() 
recombined_granulo.head()

depth_from  depth_to    z_1000     z_710     z_500  \
hole_id  code_geol                                                       
LBU_01_1 GZ              34.08     45.00  0.000000  0.036397  1.774340   
         IZ              23.94     34.08  0.050013  0.025006  0.425106   
         TZ              18.00     23.94  0.133356  0.166694  1.200200   
LBU_01_2 IZ              27.25     35.00  0.000000  0.000000  0.225169   
         TZ              19.00     27.25  0.000000  0.000000  0.455961   

                       z_355      z_250      z_180      z_125       z_90  \
hole_id  code_geol                                                         
LBU_01_1 GZ         8.999090  24.040036  43.712466  16.669700   3.803458   
         IZ         4.909561  27.315162  50.362591  12.503126   2.784029   
         TZ         5.317553  16.236039  23.637273  20.453409  25.354226   
LBU_01_2 IZ         3.727796  30.597948  52.601951  11.446085   0.950713   
         TZ         9.530694  33.084964  43.138345  11.510231   1.423488   

                        z_63       z_0        afa         ags  \
hole_id  code_geol                                              
LBU_01_1 GZ         0.682439  0.282075  60.909091  246.090909   
         IZ         1.450363  0.175044  61.416667  238.416667   
         TZ         6.934489  0.566761  83.333333  197.000000   
LBU_01_2 IZ         0.262697  0.187641  58.625000  241.625000   
         TZ         0.578292  0.278025  57.222222  255.777778   

                    theor_spec_surface  fineness_modulus  clay  
hole_id  code_geol                                              
LBU_01_1 GZ                        NaN               NaN   NaN  
         IZ                        NaN               NaN   NaN  
         TZ                        NaN               NaN   NaN  
LBU_01_2 IZ                        NaN               NaN   NaN  
         TZ                        NaN               NaN   NaN

In [36]:
writer = pd.ExcelWriter("../_RESULTS/MHZ_processed_overview.xlsx")

df_MHZ_collar.set_index("hole_id").to_excel(writer, "MHZ_collar_combined")
recombined_granulo.to_excel(writer, "MHZ_granulo_recombined", merge_cells=False)

writer.save()

In [37]:
# Set hole_id as index level 1 and code_geol as index level 2 and save individual dataframes with coordinates data
writer2 = pd.ExcelWriter("../_RESULTS/MHZ_processed_individual.xlsx")

subsets_MHZ_averages_granulo_norm_indexed = {}
for cat, df in subsets_MHZ_averages_granulo_norm.items():
    subsets_MHZ_averages_granulo_norm_indexed[cat] = df.set_index(["hole_id", "code_geol"])
    df_merge = pd.merge(df_MHZ_collar, df, left_on="hole_id", right_on="hole_id").set_index("hole_id")
    df_merge.to_excel(writer2, cat)

writer2.save()

___

### 6.. MHZ granulo backup

In [31]:
df_MHZ_granulo_backup_without_codes = pd.read_csv(path_MHZ + "MHZ_granulo_backup_without_codes.csv", sep=";")
df_MHZ_granulo_backup_without_codes.drop(["code_geol"], inplace=True, axis=1)
df_MHZ_granulo_backup_without_codes.tail()

,hole_id,depth_from,depth_to,samp_id,z_1000,z_710,z_500,z_355,z_250,z_180,z_125,z_90,z_63,z_0,afa,ags,theor_spec_surface,fineness_modulus,clay,Comment
2287,MHZ_16_08,30.0,31,PR-016635-066,0.0,0.0,0.5,2.1,4.3,31.6,53.3,7.0,0.7,0.4,75.0,182.0,137.0,1.00,0.44,NaN
2288,MHZ_16_08,31.0,32,PR-016635-067,0.0,0.0,0.1,0.6,2.7,27.1,58.5,9.4,1.0,0.4,79.0,170.0,143.0,0.93,0.40,NaN
2289,MHZ_16_08,32.0,33,PR-016635-068,0.0,0.0,0.3,2.0,3.8,29.5,55.3,8.0,0.8,0.3,77.0,179.0,139.0,0.97,0.68,NaN
2290,MHZ_16_08,33.0,34,PR-016635-069,0.0,0.0,0.0,1.0,4.3,28.7,56.8,8.0,0.8,0.3,77.0,175.0,140.0,0.96,0.51,NaN
2291,MHZ_16_08,34.0,35,PR-016635-070,0.0,0.0,0.1,1.6,4.2,28.0,56.1,8.7,0.9,0.4,78.0,176.0,141.0,0.96,0.81,NaN


___

### 7. MHZ washed

In [32]:
df_MHZ_washed = pd.read_csv(path_MHZ + "MHZ_washed.csv", sep=";")
df_MHZ_washed.tail()

,hole_id,depth_from,depth_to,samp_id,fe_w,al_w,ti_w,k_w,ca_w,mg_w,na_w,si_w,wit_w,wit_ind_w,comment
55,LBU_86_5,29.0,30.0,NaN,0.01,0.08,0.03,NaN,NaN,NaN,NaN,NaN,NaN,39.9,NaN
56,LBU_86_5,30.0,31.0,NaN,0.01,0.03,0.02,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN
57,LBU_86_5,31.0,33.0,NaN,0.01,0.07,0.03,NaN,NaN,NaN,NaN,NaN,NaN,38.5,NaN
58,LBU_86_5,33.0,34.0,NaN,0.01,0.08,0.03,NaN,NaN,NaN,NaN,NaN,NaN,37.0,NaN
59,LBU_86_5,34.0,35.0,NaN,0.01,0.07,0.03,NaN,NaN,NaN,NaN,NaN,NaN,32.3,NaN


In [33]:
df_MHZ_washed.isnull().sum()

hole_id        0
depth_from     0
depth_to       0
samp_id       60
fe_w           0
al_w           0
ti_w           0
k_w           37
ca_w          37
mg_w          60
na_w          60
si_w          60
wit_w         60
wit_ind_w      0
comment       60
dtype: int64

* fe_w : $Fe_{2}O_{3}$
* al_w : $Al_{2}O_{3}$
* ti_w : $TiO_{2}$
* k_w : $K_{2}O$
* ca_w : $CaO$
* mg_w : $MgO$
* na_w : $Na_{2}O$
* si_w : $SiO_{2}$
* wit_w : ?
* wit_ind_w : ?

___

### 8. MHZ styles

In [14]:
df_MHZ_styles = pd.read_csv(path_MHZ + "MHZ_styles.csv", sep=";")
df_MHZ_styles.tail()

,code,field_name,from_value,graphics_colour,graphics_pattern,line_colour,line_style,line_weight,marker_size,marker_style,plotting_colour,plotting_pattern,style_type,table_name,to_value
71,NaN,OD,60.0,r=1.0 g=0.6 b=0.0,solid,periwinkle,|--->,1,0.5,/.\,r=1.0 g=0.6 b=0.0,solid,N,float,85.0
72,NaN,OD,85.0,yellow,solid,periwinkle,|--->,1,0.5,/.\,yellow,solid,N,float,90.0
73,NaN,OD,90.0,r=0.4 g=0.8 b=0.0,solid,periwinkle,|--->,1,0.5,/.\,r=0.4 g=0.8 b=0.0,solid,N,float,99.0
74,NaN,z_250,0.0,periwinkle,solid,periwinkle,|--->,1,0.5,/.\,periwinkle,solid,N,granulo,10.0
75,NaN,z_250,10.0,r=0.2 g=1.0 b=0.0,solid,periwinkle,|--->,1,0.5,/.\,r=0.2 g=1.0 b=0.0,solid,N,granulo,20.0


In [29]:
df_MHZ_washed["hole_id"].unique()

array(['LBU_81_1', 'LBU_81_2', 'LBU_81_3', 'LBU_81_4', 'LBU_81_5',
       'LBU_81_6', 'LBU_86_4', 'LBU_86_5'], dtype=object)

In [30]:
set(df_MHZ_washed["hole_id"]) & set(df_MHZ_granulo["hole_id"])

{'LBU_81_1', 'LBU_81_2', 'LBU_81_3', 'LBU_81_5', 'LBU_81_6'}